In [2]:
# Import neccesary libraries
import tensorflow as tf
import pandas as pd
import numpy as np
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs
from typing import Dict, Text

In [3]:
ratings = tfds.load("movielens/100k-ratings", split = "train")
movie = tfds.load("movielens/100k-movies")

2022-08-16 17:00:09.531460: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
ratings = ratings.map(lambda x: {
    "movie_title" : x["movie_title"],
    "user_id" : x["user_id"],
    "user_rating" : x["user_rating"] # We are going to keep the ratings, they are considered explicit feedback
})

In [5]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100000, seed = 42, reshuffle_each_iteration=False)

train = shuffled.take(80000)
test = shuffled.skip(80000).take(20000)

In [6]:
movie_titles = ratings.batch(1000000).map(lambda x: x["movie_title"])
user_ids = ratings.batch(1000000).map(lambda x :x ["user_id"])

unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

In [27]:
class RankingModel(tf.keras.Model):

  def __init__(self):
    super().__init__()
    embedding_dimension = 32

    # Compute embeddings for users.
    self.user_embeddings = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_user_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
    ])

    # Compute embeddings for movies.
    self.movie_embeddings = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_movie_titles, mask_token=None),
      tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
    ])

    # Compute predictions.
    self.ratings = tf.keras.Sequential([
      # Learn multiple dense layers.
      tf.keras.layers.Dense(256, activation="relu"),
      tf.keras.layers.Dense(64, activation="relu"),
      # Make rating predictions in the final layer.
      tf.keras.layers.Dense(1)
  ])

  def call(self, inputs):

    user_id, movie_title = inputs

    user_embedding = self.user_embeddings(user_id)
    movie_embedding = self.movie_embeddings(movie_title)

    return self.ratings(tf.concat([user_embedding, movie_embedding], axis=1))

In [28]:
RankingModel()((["42"], ["One Flew Over the Cuckoo's Nest (1975)"]))

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[-0.02926024]], dtype=float32)>

In [29]:
task = tfrs.tasks.Ranking(loss = tf.keras.losses.MeanSquaredError(),
                          metrics = [tf.keras.metrics.RootMeanSquaredError()])

                          # The task itself is a Keras layer that takes true and predicted as arguments, and returns the computed loss. We'll use that to implement the model's training loop.



In [30]:
class MovielensModel(tfrs.models.Model):

  def __init__(self):
    super().__init__()
    self.ranking_model: tf.keras.Model = RankingModel()
    self.task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
      loss = tf.keras.losses.MeanSquaredError(),
      metrics=[tf.keras.metrics.RootMeanSquaredError()]
    )

  def call(self, features: Dict[str, tf.Tensor]) -> tf.Tensor:
    return self.ranking_model(
        (features["user_id"], features["movie_title"]))

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    labels = features.pop("user_rating")

    rating_predictions = self(features)

    # The task computes the loss and the metrics.
    return self.task(labels=labels, predictions=rating_predictions)

In [31]:
model = MovielensModel()
model.compile(optimizer = tf.keras.optimizers.Adagrad(learning_rate=0.1))

cached_train = train.shuffle(100000).batch(8192).cache()
cached_test = test.batch(4096).cache()

In [32]:
model.fit(cached_train, epochs = 3)

Epoch 1/3
10/10 [==============================] - 2s 34ms/step - root_mean_squared_error: 2.0978 - loss: 4.0680 - regularization_loss: 0.0000e+00 - total_loss: 4.0680
Epoch 2/3
10/10 [==============================] - 0s 20ms/step - root_mean_squared_error: 1.1650 - loss: 1.3474 - regularization_loss: 0.0000e+00 - total_loss: 1.3474
Epoch 3/3
10/10 [==============================] - 0s 17ms/step - root_mean_squared_error: 1.1172 - loss: 1.2461 - regularization_loss: 0.0000e+00 - total_loss: 1.2461
